# Privilege Escalation: Security Group Breach Prevention

## Overview

- **Domain**: Security & Infrastructure
- **Risk Level**: Critical
- **Time**: 10-15 minutes
- **Prevents**: Data breaches, privilege escalation, unauthorized access

## Real-World Context

Privilege escalation attacks are among the most dangerous security incidents. An agent with limited permissions attempts to perform actions beyond its authorization—like opening SSH ports to the internet. This scenario demonstrates how role-based access control (RBAC) in Relay enforces the principle of least privilege.

## What You'll Learn

- How agents can exceed their intended authorization
- Why role-based policies are critical for security
- How Zero Trust principles apply to agent systems
- Security incident response from audit trails

In [1]:
import sys
sys.path.insert(0, '../')

import requests
import json
from datetime import datetime

# Verify Relay is running
try:
    requests.get('http://localhost:8181/health', timeout=2)
    print('✅ Relay infrastructure is running')
except:
    print('❌ Start infrastructure: cd infra && docker-compose up -d')
    raise

✅ Relay infrastructure is running


## The Scenario

A junior support agent with view-only permissions attempts to modify production security groups—potentially opening SSH access to the entire internet. This could be due to social engineering, a bug, or compromise.

In [2]:
agent_info = {
    "agent_id": "junior-support-agent-042",
    "role": "support",
    "authorized_actions": [
        "view_tickets",
        "respond_to_customers",
        "create_refunds (≤$100)"
    ],
    "NOT_authorized": [
        "modify_infrastructure",
        "access_production_systems",
        "change_security_settings"
    ]
}

print("👤 Agent Profile:")
print(json.dumps(agent_info, indent=2))

👤 Agent Profile:
{
  "agent_id": "junior-support-agent-042",
  "role": "support",
  "authorized_actions": [
    "view_tickets",
    "respond_to_customers",
    "create_refunds (\u2264$100)"
  ],
  "NOT_authorized": [
    "modify_infrastructure",
    "access_production_systems",
    "change_security_settings"
  ]
}


## Without Relay: Security Disaster

Without proper access controls, the agent can execute privileged operations.

In [3]:
print("❌ WITHOUT RELAY:")
print("─" * 80)

print("\n🚨 Agent attempts unauthorized action:")
malicious_action = """
aws.modify_security_group(
    group_id='sg-prod-web-servers',
    add_rule={
        'protocol': 'tcp',
        'port': 22,
        'source': '0.0.0.0/0'  # Open SSH to entire internet!
    }
)
"""
print(malicious_action)

print("\n💀 Security Disaster Unfolds:")
disaster_timeline = [
    "T+0min: SSH port 22 opened to 0.0.0.0/0",
    "T+5min: Automated scanners detect open port",
    "T+15min: Brute force attacks begin",
    "T+2hrs: Weak password compromised",
    "T+4hrs: Lateral movement through network",
    "T+8hrs: Customer database accessed",
    "T+12hrs: Data exfiltration begins"
]

for event in disaster_timeline:
    print(f"   {event}")

print("\n💰 Impact:")
print("   └─ Complete data breach")
print("   └─ Average cost: $4.45M (IBM 2023)")
print("   └─ Customer trust destroyed")
print("   └─ Company reputation ruined")

❌ WITHOUT RELAY:
────────────────────────────────────────────────────────────────────────────────

🚨 Agent attempts unauthorized action:

aws.modify_security_group(
    group_id='sg-prod-web-servers',
    add_rule={
        'protocol': 'tcp',
        'port': 22,
        'source': '0.0.0.0/0'  # Open SSH to entire internet!
    }
)


💀 Security Disaster Unfolds:
   T+0min: SSH port 22 opened to 0.0.0.0/0
   T+5min: Automated scanners detect open port
   T+15min: Brute force attacks begin
   T+2hrs: Weak password compromised
   T+4hrs: Lateral movement through network
   T+8hrs: Customer database accessed
   T+12hrs: Data exfiltration begins

💰 Impact:
   └─ Complete data breach
   └─ Average cost: $4.45M (IBM 2023)
   └─ Customer trust destroyed
   └─ Company reputation ruined


## With Relay: Role-Based Protection

Relay enforces role-based access control, preventing privilege escalation.

In [4]:
print("✅ WITH RELAY:")
print("─" * 80)

# Build the action manifest
manifest = {
    "agent": {
        "agent_id": "junior-support-agent-042",
        "role": "support"
    },
    "action": {
        "provider": "aws",
        "method": "modify_security_group",
        "parameters": {
            "group_id": "sg-prod-web-servers",
            "add_rule": {
                "protocol": "tcp",
                "port": 22,
                "source": "0.0.0.0/0"
            }
        }
    }
}

print("\n🛡️  Role-Based Policy Check:")
print("   Policy: Only 'security-engineer' role can modify security groups")
print("   Agent role: support")
print("   └─ 🚫 BLOCKED: Insufficient privileges")

print("\n🔒 Zero Trust Principle Enforced:")
print("   ✅ Never trust, always verify")
print("   ✅ Least privilege by default")
print("   ✅ Explicit allow required")
print("   ✅ Every action authenticated and authorized")

✅ WITH RELAY:
────────────────────────────────────────────────────────────────────────────────

🛡️  Role-Based Policy Check:
   Policy: Only 'security-engineer' role can modify security groups
   Agent role: support
   └─ 🚫 BLOCKED: Insufficient privileges

🔒 Zero Trust Principle Enforced:
   ✅ Never trust, always verify
   ✅ Least privilege by default
   ✅ Explicit allow required
   ✅ Every action authenticated and authorized


## Security Incident Response

The audit trail captures this attack attempt for immediate response.

In [5]:
security_incident = {
    "timestamp": datetime.now().isoformat(),
    "severity": "CRITICAL",
    "incident_type": "PRIVILEGE_ESCALATION_ATTEMPT",
    "agent": {
        "id": "junior-support-agent-042",
        "role": "support",
        "authorized_level": "read-only"
    },
    "attempted_action": {
        "provider": "aws",
        "method": "modify_security_group",
        "target": "sg-prod-web-servers",
        "required_role": "security-engineer"
    },
    "decision": "DENIED",
    "reason": "Insufficient privileges - role mismatch",
    "risk_indicators": [
        "Attempting infrastructure modification",
        "Target: production security group",
        "Would expose SSH to internet",
        "Agent role: support (unauthorized)"
    ],
    "response_actions": [
        "Blocked action immediately",
        "Alert security team",
        "Notify agent owner/manager",
        "Review agent behavior history",
        "Investigate compromise possibility"
    ]
}

print("🚨 Security Incident Report:")
print(json.dumps(security_incident, indent=2))

🚨 Security Incident Report:
{
  "timestamp": "2026-01-18T02:19:16.162935",
  "severity": "CRITICAL",
  "incident_type": "PRIVILEGE_ESCALATION_ATTEMPT",
  "agent": {
    "id": "junior-support-agent-042",
    "role": "support",
    "authorized_level": "read-only"
  },
  "attempted_action": {
    "provider": "aws",
    "method": "modify_security_group",
    "target": "sg-prod-web-servers",
    "required_role": "security-engineer"
  },
  "decision": "DENIED",
  "reason": "Insufficient privileges - role mismatch",
  "risk_indicators": [
    "Attempting infrastructure modification",
    "Target: production security group",
    "Would expose SSH to internet",
    "Agent role: support (unauthorized)"
  ],
  "response_actions": [
    "Blocked action immediately",
    "Alert security team",
    "Notify agent owner/manager",
    "Review agent behavior history",
    "Investigate compromise possibility"
  ]
}


## Try It With Curl

Test role-based access control without any installation:

```bash
# Blocked: Support agent cannot modify security groups
curl -X POST http://localhost:8181/v1/data/relay/policies/main \
  -H "Content-Type: application/json" \
  -d '{
    "input": {
      "agent": {
        "agent_id": "junior-support-agent",
        "role": "support"
      },
      "action": {
        "provider": "aws",
        "method": "modify_security_group",
        "parameters": {"group_id": "sg-prod"}
      }
    }
  }'

# Approved: Security engineer can modify security groups
curl -X POST http://localhost:8181/v1/data/relay/policies/main \
  -H "Content-Type: application/json" \
  -d '{
    "input": {
      "agent": {
        "agent_id": "security-engineer-001",
        "role": "security-engineer"
      },
      "action": {
        "provider": "aws",
        "method": "modify_security_group",
        "parameters": {"group_id": "sg-prod"}
      }
    }
  }'
```

## Policy Implementation

Here's the policy that prevents this scenario:

```yaml
policies:
  - name: "infrastructure_rbac"
    description: "Role-based access control for infrastructure changes"
    rules:
      # Allow security engineers to modify security groups
      - id: "security_engineer_access"
        condition:
          provider: "aws"
          method: "modify_security_group"
          agent:
            role_in: ["security-engineer", "admin"]
        action: "allow"
        reason: "Authorized role for security group modification"
      
      # Deny all other roles
      - id: "deny_unauthorized_security_changes"
        condition:
          provider: "aws"
          method: "modify_security_group"
        action: "deny"
        reason: "Only security engineers can modify security groups"
        alert:
          severity: "critical"
          channels: ["pagerduty", "security-team"]
          message: "Privilege escalation attempt detected"
      
      # Additional infrastructure protections
      - id: "restrict_production_access"
        condition:
          provider: "aws"
          method_pattern: "^(delete|terminate|modify)_.*"
          resource:
            environment: "production"
          agent:
            role_not_in: ["admin", "sre", "security-engineer"]
        action: "deny"
        reason: "Production modifications require elevated privileges"
```

**Zero Trust Principles:**

1. **Never Trust, Always Verify**: Every action requires authorization
2. **Least Privilege**: Minimal permissions by default
3. **Assume Breach**: Agent compromise should not lead to escalation
4. **Verify Explicitly**: Check role on every request

**To implement this policy:**

1. Add to `/policies/infrastructure_security.yaml`
2. Define role hierarchy and permissions matrix
3. Configure critical incident alerting
4. Run: `python scripts/bootstrap_policies.py`
5. Restart Relay gateway
6. Test with both authorized and unauthorized roles
7. Document escalation procedures for security team

## Key Takeaways

- **Least privilege is mandatory** - agents should only have minimum necessary permissions
- **Roles prevent escalation** - even compromised agents can't exceed authorization
- **Zero Trust for agents** - never assume an agent's intentions are safe
- **Audit trails enable forensics** - complete history of authorization attempts
- **Defense in depth** - Relay is one layer in comprehensive security architecture

## Next Steps

- **Review**: [05_real_world_scenarios.ipynb](05_real_world_scenarios.ipynb) - Cost-benefit analysis
- **Learn**: [04_company_policies.ipynb](04_company_policies.ipynb) - RBAC policy patterns
- **Explore**: [02_adversarial_prompt_protection.ipynb](02_adversarial_prompt_protection.ipynb) - More security scenarios